In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
import xgboost as xgb
import torch
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
project = "drive/My Drive/Assignment"
train = pd.read_csv(project + "/Train.csv")
train_sample = train.sample(n=30000,random_state=42)

<ipython-input-19-3363561198>:2: DtypeWarning: Columns (13,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(project + "/Train.csv")


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
X_train = train_sample.drop(["SalePrice"], axis=1)
y_train = train_sample["SalePrice"]
sample = X_train.head(3)
sample1= sample.iloc[:1]
sample2=sample.iloc[2:3]


In [ ]:
for col in X_train.columns:
    if (sample1[col].isna().item() or sample2[col].isna().item()):
      X_train = X_train.drop([col], axis=1)
f_features = X_train.columns
len(f_features)

16

In [ ]:
X_val = pd.read_csv(project + "/Valid.csv", usecols= f_features)
y_val = pd.read_csv(project+"/median_benchmark.csv",usecols=['SalePrice'])
y_val = np.log(y_val["SalePrice"])
test = pd.read_csv(project + "/Test.csv", usecols= f_features)

In [ ]:
categorical_cols = X_train.select_dtypes(include='object').columns
categorical_cols
for i in categorical_cols:
  X_train[i]=X_train[i].astype('category')
  X_val[i]=X_val[i].astype('category')
  test[i]=test[i].astype('category')


In [ ]:
for name, column in test.items():
  if column.dtype.name == 'category':
    X_train[name] = column.cat.codes
    test[name] = column.cat.codes
X_train.fillna(X_train.median(), inplace = True)
test.dtypes.value_counts()

,count
int64,6
int8,6
int16,3
float64,1


In [ ]:
X_train.shape , X_val.shape , test.shape

((30000, 16), (11573, 16), (12457, 16))

In [ ]:
model = RandomForestRegressor(n_estimators=1, bootstrap=False, random_state=42)
model.fit(X_train,y_train)
y_pred= model.predict(test)

In [ ]:
output = pd.DataFrame({'id': test.SalesID,
                       'yield': y_pred})
output.to_csv('submission.csv', index=False)